In [ ]:
import torch
import torchvision
import numpy as np
import random

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
ROOT = '.data'

train_data = torchvision.datasets.MNIST(root=ROOT, train=True, download=True)

In [ ]:
mean = train_data.data.float().mean() / 225
std = train_data.data.float().std() / 225

print(f'data mean: {mean}')
print(f'data std: {std}')

In [ ]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.RandomRotation(5, fill=(0, )),
    transforms.RandomCrop(28, padding=2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std])
])

In [ ]:
train_data = torchvision.datasets.MNIST(root=ROOT,
                                        train=True,
                                        download=True,
                                        transform=train_transforms)

test_data = torchvision.datasets.MNIST(root=ROOT,
                                       train=False,
                                       download=True,
                                       transform=test_transforms)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
import matplotlib.pyplot as plt

def plot_images(images):
    n_images = len(images)
    rows = cols = int(np.sqrt(n_images))

    fig = plt.figure()
    for i in range(rows * cols):
        ax = fig.add_subplot(rows, cols, i+1)
        ax.imshow(images[i].view(28, 28).cpu().numpy(), cmap='bone')
        ax.axis('off')

In [ ]:
N_IMAGES = 25

# images = [image for image, label in [train_data[i] for i in range(N_IMAGES)]]
images = [train_data[i][0] for i in range(N_IMAGES)]

plot_images(images)

In [ ]:
VALID_RATIO = 0.1

n_train_examples = int(len(train_data) * (1 - VALID_RATIO))
n_test_examples = int(len(train_data) * VALID_RATIO)

In [ ]:
train_data, valid_data = torch.utils.data.random_split(train_data, [n_train_examples, n_test_examples])

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

In [ ]:
import copy

valid_data = copy.deepcopy(valid_data)
valid_data.dataset.transform = test_transforms

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 256

train_iterator = DataLoader(train_data, BATCH_SIZE, True)
valid_iterator = DataLoader(valid_data, BATCH_SIZE, False)
test_iterator = DataLoader(test_data, BATCH_SIZE, False)

In [ ]:
from torch import nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc_1 = nn.Linear(784, 250)
        self.fc_2 = nn.Linear(250, 100)
        self.fc_3 = nn.Linear(100, 10)
        self.relu = nn.ReLU()
    
    def forward(self, batch):
        batch_size = batch.shape[0]

        batch = batch.view(batch_size, -1)

        h_1 = self.relu(self.fc_1(batch))
        h_2 = self.relu(self.fc_2(h_1))
        
        preds = self.fc_3(h_2)

        return preds, h_2

In [ ]:
model = MLP()

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
criterion.to(device)

In [ ]:
def calculate_accuracy(preds, labels):
    top_preds = torch.argmax(preds, dim=1, keepdim=True)
    correct = top_preds.eq(labels.view_as(top_preds)).sum()
    acc = correct.float() / labels.shape[0]

    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for images, labels in iterator:
        images, labels = images.to(device), labels.to(device)

        preds, _ = model(images)

        loss = criterion(preds, labels)
        acc = calculate_accuracy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for images, labels in iterator:
            images, labels = images.to(device), labels.to(device)

            preds, _ = model(images)

            loss = criterion(preds, labels)
            acc = calculate_accuracy(preds, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
N_EPOCH = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCH):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, train_iterator, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'mlp-model')
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('mlp-model'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'\t Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

In [ ]:
import torch.nn.functional as F

def get_prediction(model, iterator, device):
    model.eval()

    images, labels, preds = [], [], []

    with torch.no_grad():
        for image, label in iterator:
            image = image.to(device)
            
            pred, _ = model(image)
            pred = F.softmax(pred, dim=-1)
            # pred = torch.argmax(pred, dim=-1)

            images.append(image.cpu())
            labels.append(label)
            preds.append(pred.cpu())
    
    images = torch.cat(images, dim=0)
    labels = torch.cat(labels, dim=0)
    preds = torch.cat(preds, dim=0)

    return images, labels, preds

In [ ]:
from sklearn import metrics

def plot_confusion_matrix(labels, preds):
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot(1, 1, 1)
    cm = metrics.confusion_matrix(labels, preds)
    cm = metrics.ConfusionMatrixDisplay(cm, display_labels = range(10))
    cm.plot(values_format = 'd', cmap = 'Blues', ax = ax)

In [ ]:
images, labels, preds = get_prediction(model, test_iterator, device)

plot_confusion_matrix(labels, torch.argmax(preds, dim=1))

In [ ]:
def plot_most_incorrect(incorrect_examples, n_images):

    rows = int(np.sqrt(n_images))
    cols = int(np.sqrt(n_images))

    fig = plt.figure(figsize = (20, 10))
    for i in range(rows*cols):
        ax = fig.add_subplot(rows, cols, i+1)
        image, true_label, probs = incorrect_examples[i]
        true_prob = probs[true_label]
        incorrect_prob, incorrect_label = torch.max(probs, dim=0)
        ax.imshow(image.view(28, 28).cpu().numpy(), cmap='bone')
        ax.set_title(f'true label: {true_label} ({true_prob:.3f})\n' \
                     f'pred label: {incorrect_label} ({incorrect_prob:.3f})')
        ax.axis('off')
    fig.subplots_adjust(hspace= 0.5)

In [ ]:
corrects = torch.eq(labels, torch.argmax(preds, dim=1))

incorrect_examples = []
for image, label, pred, correct in zip(images, labels, preds, corrects):
    if not correct:
        incorrect_examples.append((image, label, pred))

incorrect_examples.sort(reverse=True, key=lambda x: torch.max(x[2], dim=0).values)

In [ ]:
N_IMAGES = 25

plot_most_incorrect(incorrect_examples, N_IMAGES)